In [1]:
!pip install lancedb

In [2]:
!pip install tantivy

### Required libraries

In [86]:
import lancedb
import pandas as pd
import pyarrow as pa
import sys,os
module_path = os.path.abspath(os.getcwd() + '../../')
sys.path.append(module_path)
print(sys.path)
os.getcwd()
from src.constants import LANCEDB_DICT, EMBEDDINGS_DICT
from sentence_transformers import SentenceTransformer
from lancedb.embeddings import get_registry
from lancedb.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from lancedb.pydantic import LanceModel, Vector

['/Users/dusisarathchandra/miniconda3/lib/python310.zip', '/Users/dusisarathchandra/miniconda3/lib/python3.10', '/Users/dusisarathchandra/miniconda3/lib/python3.10/lib-dynload', '', '/Users/dusisarathchandra/miniconda3/lib/python3.10/site-packages', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper/notebooks..', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper/notebooks..', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper', '/Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service

In [88]:
print(LANCEDB_DICT.get('uri'))

cloud_service_knowledge_base/vector_db


### Create a Database class which takes care of vector database functions

In [89]:

class lanceDB:
    def __init__(self, uri):
        self.uri = uri
        self.db = lancedb.connect(uri)

    def create_table(self, table_name, schema=None, mode="overwrite"):
        print(f"Creating table: {table_name}")
        print(f"mode is {mode}")
        return self.db.create_table(table_name, schema=schema, mode=mode)

    def insert(self, table_name, data):
        self.db.add(table_name, data)

    def query(self, query):
        return self.db.query(query)

    def close(self):
        self.db.close()
        self.async_db.close()
    def table_names(self):
        return self.db.table_names()


### Initiate LanceDB Connection

In [90]:
db = lanceDB(uri=LANCEDB_DICT.get('uri'))
print(db.table_names())


[]


### Locally Load the sentence transformer model

In [91]:
model_name = EMBEDDINGS_DICT.get('model_name')
device = EMBEDDINGS_DICT.get('device')
model = SentenceTransformer(model_name, device=device)

/Users/dusisarathchandra/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Verfiy the embeddings

In [92]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
embeddings[0]


array([ 6.76568672e-02,  6.34958893e-02,  4.87131067e-02,  7.93049783e-02,
        3.74480560e-02,  2.65278690e-03,  3.93749177e-02, -7.09835812e-03,
        5.93614504e-02,  3.15369740e-02,  6.00980707e-02, -5.29051721e-02,
        4.06068228e-02, -2.59308200e-02,  2.98427045e-02,  1.12690218e-03,
        7.35148787e-02, -5.03818206e-02, -1.22386619e-01,  2.37028264e-02,
        2.97266264e-02,  4.24768180e-02,  2.56337617e-02,  1.99509831e-03,
       -5.69191240e-02, -2.71598697e-02, -3.29036377e-02,  6.60248548e-02,
        1.19007245e-01, -4.58791889e-02, -7.26215094e-02, -3.25840972e-02,
        5.23412935e-02,  4.50552851e-02,  8.25296901e-03,  3.67024280e-02,
       -1.39415357e-02,  6.53919131e-02, -2.64271721e-02,  2.06365905e-04,
       -1.36643983e-02, -3.62809487e-02, -1.95043124e-02, -2.89738476e-02,
        3.94270532e-02, -8.84091482e-02,  2.62422091e-03,  1.36713963e-02,
        4.83062230e-02, -3.11565585e-02, -1.17329203e-01, -5.11690043e-02,
       -8.85288417e-02, -

In [93]:
# dimensions of the vector embeddings
len(embeddings[0])

384

In [94]:
print(EMBEDDINGS_DICT.get('vector_dims')) # setting this to default under constants

384


### Load LanceDB embedder

In [95]:
# calling the lanceDB embedding functions in the registry
# Using `sentence-transformers` as the model registry
model_registry = get_registry().get("sentence-transformers")
model = model_registry.create(name=model_name, device=device)

### Define the schema for the table

In [96]:
class CloudServiceInestionDataModel(LanceModel):
    description: str = model.SourceField()
    vector_description: Vector(dim=model.ndims()) = model.VectorField()
    cloud_provider: str
    service: str
    category: str =  model.SourceField()
    class Config:
        arbitrary_types_allowed = True

#### Read the dataset as a Pandas Dataframe

In [97]:
df = pd.read_csv('../data/Cloud_Provider_Services.csv')

In [98]:
df.tail()

,cloud_provider,category,service,description
1297,oci,Network,Network,Network services
1298,oci,Network,Load balancers,Load balancer service
1299,oci,Network,Virtual cloud networks,Virtual cloud networks
1300,oci,Network,Dns management,DNS management service
1301,oci,Network,Fast connect,Fast connect service


#### Create a list of documents to be added to LanceDB table

In [99]:
data = df.apply(
    lambda row: {
        "category": row["category"],
        "cloud_provider": row["cloud_provider"],
        "service": row["service"],
        "description": row["description"],
    },
    axis=1,
).values.tolist()

In [100]:
data[0]

{'category': 'Analytics',
 'cloud_provider': 'aws',
 'service': 'Amazon Athena',
 'description': 'An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.'}

#### Create a lanceDB table with the provided schema model

In [101]:
table = db.create_table("vector_db", schema=CloudServiceInestionDataModel, mode="overwrite")

Creating table: vector_db
mode is overwrite


[2024-09-05T13:26:22Z WARN  lance::dataset] No existing dataset at /Users/dusisarathchandra/Documents/Projects/workspaces/llmworkspace/cloud-service-helper/notebooks/cloud_service_knowledge_base/vector_db/vector_db.lance, it will be created


#### Add the data to the table

In [103]:
table.add(data)

### Full-text search 
[LanceDB - full Text Search documentation](https://lancedb.github.io/lancedb/fts/)

In [104]:
table.create_fts_index("description", replace=True)

In [105]:
#query = "An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL."
query = "query service that makes it easy to analyze data"
fts_actual = table.search(query, query_type="fts").limit(10).to_pandas()

In [44]:
fts_actual.head(10)

,description,vector_description,cloud_provider,service,category,score
0,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,17.903425
1,A web service that makes it easy to process la...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Elastic MapReduce,Analytics,13.446853
2,A service that makes it easy to add image and ...,"[-0.024350997, 0.016706645, 0.037720546, -0.00...",aws,Amazon Rekognition,Artificial Intelligence,12.409848
3,A fully managed service that makes it easy to ...,"[0.0020500543, 0.13870272, -0.05612806, -0.001...",aws,Amazon Managed Streaming for Apache Kafka,Compute,12.025981
4,A machine learning service that makes it easy ...,"[-0.024350997, 0.016706645, 0.037720546, -0.00...",aws,Amazon Personalize,Artificial Intelligence,10.932827
5,A fast cloud-powered business intelligence ser...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon QuickSight,Analytics,10.621030
6,A fully managed extract transform and load (ET...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,AWS Glue,Analytics,10.530128
7,A fully managed service that makes it easy to ...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,AWS IoT Analytics,Analytics,10.530128
8,A service that makes it easy for developers of...,"[-0.024350997, 0.016706645, 0.037720546, -0.00...",aws,Amazon Machine Learning,Artificial Intelligence,10.489511
9,A fully managed service that makes it easy for...,"[0.0020500543, 0.13870272, -0.05612806, -0.001...",aws,Amazon API Gateway,Compute,10.080750


### Vector Search
[Vector Arithmetic with LanceDB: An Intro to Vector Embeddings](https://blog.lancedb.com/vector-arithmetic-with-lancedb-an-intro-to-vector-embeddings/)

In [106]:
query = "query that makes it easy to analyze S3"
vector_actual = table.search(query, query_type="vector").limit(10).to_pandas()

In [107]:
vector_actual.head(5)

## Expected answer: `Amazon Athena` which is a query service that makes it easy to analyze data in Amazon S3 using standard SQL.

,description,vector_description,cloud_provider,service,category,_distance
0,A managed service in the AWS Cloud that makes ...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon CloudSearch,Analytics,1.244212
1,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,1.244212
2,A web service that makes it easy to process la...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Elastic MapReduce,Analytics,1.244212
3,A platform on AWS to collect process and analy...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Kinesis,Analytics,1.244212
4,Logging service,"[0.00044248256, -0.0026783892, -0.12139637, 0....",oci,Logging,Analytics,1.244212


### Hybrid Search 

[Example for Hybrid Search from LanceDB github](https://github.com/lancedb/vectordb-recipes/blob/main/examples/Inbuilt-Hybrid-Search/Inbuilt_Hybrid_Search_with_LanceDB.ipynb)

In [109]:
from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(
    weight=0.8
)  # Weight = 0 Means pure Text Search (BM-25) and 1 means pure Sementic (Vector) Search
query = "query that makes it easy to analyze data"
table.search(
    query,
    query_type="hybrid",
    vector_column_name="vector_description",
).rerank(reranker=reranker).limit(5).to_pandas()

,description,vector_description,cloud_provider,service,category,_relevance_score
0,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,0.8
1,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,0.8
2,Database services,"[0.04617132, -0.020743694, -0.09271921, 0.0216...",democloud,Database,Database,0.8
3,Managed MongoDB service,"[0.04617132, -0.020743694, -0.09271921, 0.0216...",alicloud,ApsaraDB for MongoDB,Database,0.8
4,Relational database service,"[0.04617132, -0.020743694, -0.09271921, 0.0216...",alicloud,Relational Database Service (RDS),Database,0.8


In [ ]:
!pip freeze list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


accelerate==0.34.0
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
altair==5.4.1
anaconda-anon-usage @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_3eler6mjxh/croot/anaconda-anon-usage_1710965076906/work
annotated-types @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1fa2djihwb/croot/annotated-types_1709542925772/work
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1717693030552/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1707233003401/work
archspec @ file:///croot/archspec_1709217642129/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1695386808436/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru==2.0.4
async-timeou